In [67]:
from pathlib import Path
import pandas as pd

In [68]:
path = Path('./submissions/submission (18).csv')
df = pd.read_csv(path)
print(path.name)

submission (18).csv


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   patientId         3000 non-null   object
 1   PredictionString  956 non-null    object
dtypes: object(2)
memory usage: 47.0+ KB


In [70]:
df['PredictionString'].isna().sum()

2044

In [71]:
df[df['PredictionString'].notna()]

,patientId,PredictionString
10,002bcde0-d8da-4931-ab04-5d724e30261b,0.38 739.21 657.72 322.14 427.13
13,00330f7f-d114-4eb2-9c6e-558eeb3084a1,0.47 641.45 574.16 298.30 533.81 0.43 261.53 5...
15,003d17f0-bd8a-485c-bc8b-daec33f53efa,0.25 309.58 467.15 208.20 249.39
18,003fbda2-ba55-4714-a03a-83f15bec19e4,0.45 746.05 476.70 283.06 492.72 0.41 374.36 5...
21,0048e89e-f88a-41a1-b79f-ab4e7135627e,0.28 291.28 550.68 263.07 234.85
...,...,...
2977,c1b2f938-49b9-4fe6-8251-b6c90622907b,0.33 862.63 670.94 268.79 403.86 0.29 491.04 7...
2980,c1bea7c6-14cb-483c-b2ee-e958901f0667,0.40 253.33 563.77 228.06 500.10 0.30 641.34 5...
2988,c1d9b4c9-39cc-4b37-b95b-82d26a1b68df,0.26 244.82 601.70 169.75 177.80
2989,c1db263b-5d0e-47f7-9093-b7ad5b2a4d2a,0.32 377.28 545.79 200.55 229.58 0.29 750.60 5...


In [72]:
df['pred_count'] = df['PredictionString'].apply(lambda x: 0 if pd.isna(x) else int(len(x.split()) / 5))
df['pred_count'].value_counts().sort_index()

pred_count
0    2044
1     544
2     324
3      71
4      17
Name: count, dtype: int64

In [73]:
df_part = df.query('pred_count > 0')

In [74]:
s = df_part['PredictionString'].sample()
s

2624    0.29 668.78 658.13 219.27 370.53
Name: PredictionString, dtype: object

In [75]:
def move_coords(c: list) -> str:
    c = list(map(float, c))

    c[1] = c[1] - c[3] / 2
    c[2] = c[2] - c[4] / 2

    c = list(map(lambda s: f"{s:.2f}", c))

    return ' '.join(c)


def morph_prediction_string(s: str) -> str:
    mc_list = []

    if pd.isna(s):
        return s
    n = len(s.split()) // 5
    for i in range(n):
        coords = s.split()[i * 5:i * 5 + 5]
        moved_coords = move_coords(coords)
        mc_list.append(moved_coords)

    return ' '.join(mc_list)


In [76]:
s.values[0]

'0.29 668.78 658.13 219.27 370.53'

In [77]:
morph_prediction_string(s.values[0])

'0.29 559.14 472.87 219.27 370.53'

In [78]:
df['PredictionString'] = df['PredictionString'].apply(morph_prediction_string)

In [79]:
df

,patientId,PredictionString,pred_count
0,0000a175-0e68-4ca4-b1af-167204a7e0bc,NaN,0
1,0005d3cc-3c3f-40b9-93c3-46231c3eb813,NaN,0
2,000686d7-f4fc-448d-97a0-44fa9c5d3aa6,NaN,0
3,000e3a7d-c0ca-4349-bb26-5af2d8993c3d,NaN,0
4,00100a24-854d-423d-a092-edcf6179e061,NaN,0
...,...,...,...
2995,c1e88810-9e4e-4f39-9306-8d314bfc1ff1,0.29 608.16 145.18 184.29 291.13,1
2996,c1ec035b-377b-416c-a281-f868b7c9b6c3,NaN,0
2997,c1ef5b66-0fd7-49d1-ae6b-5af84929414b,NaN,0
2998,c1ef6724-f95f-40f1-b25b-de806d9bc39d,NaN,0


In [80]:
df.drop(columns='pred_count').to_csv(path.with_name(path.stem+'_morphed.csv'), index=False)

In [81]:
path.with_name(path.stem+'_morphed.csv')

WindowsPath('submissions/submission (18)_morphed.csv')